# Assignment 3: Predicting Mapping Penalties with ANN

In this assignment, a feed-forward artificial neural network (ANN) is implemented from scratch to predict the penalty score of a mapping between tasks and employees.




## Assignment Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib . pyplot as plt

!git clone https://github.com/tonyzrl/ann.git

# Task data: ID, Estimated Time, Difficulty, Deadline, Skill Required
tasks = [{"id": "T1", "estimated_time": 4, "difficulty": 3, "deadline": 8, "skill_required": "A"},
        {"id": "T2", "estimated_time": 6, "difficulty": 5, "deadline": 12, "skill_required": "B"},
        {"id": "T3", "estimated_time": 2, "difficulty": 2, "deadline": 6, "skill_required": "A"},
        {"id": "T4", "estimated_time": 5, "difficulty": 4, "deadline": 10, "skill_required": "C"},
        {"id": "T5", "estimated_time": 3, "difficulty": 1, "deadline": 7, "skill_required": "A"},
        {"id": "T6", "estimated_time": 8, "difficulty": 6, "deadline": 15, "skill_required": "B"},
        {"id": "T7", "estimated_time": 4, "difficulty": 3, "deadline": 9, "skill_required": "C"},
        {"id": "T8", "estimated_time": 7, "difficulty": 5, "deadline": 14, "skill_required": "B"},
        {"id": "T9", "estimated_time": 2, "difficulty": 2, "deadline": 5, "skill_required": "A"},
        {"id": "T10", "estimated_time": 6, "difficulty": 4, "deadline": 11, "skill_required": "C"},]

# Employee data: ID, Available hours, Skill level, Skills
employees = [{"id": "E1", "hours_avail": 10, "skill_level": 4, "skills": ["A", "C"]},
            {"id": "E2", "hours_avail": 12, "skill_level": 6, "skills": ["A", "B", "C"]},
            {"id": "E3", "hours_avail": 8, "skill_level": 3, "skills": ["A"]},
            {"id": "E4", "hours_avail": 15, "skill_level": 7, "skills": ["B", "C"]},
            {"id": "E5", "hours_avail": 9, "skill_level": 5, "skills": ["A", "C"]}]

fatal: destination path 'ann' already exists and is not an empty directory.


## Data Generation and Loading

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/artificial_neural_network/data/task_assignment_data.csv')

## Data Preprocessing

In [ ]:
def one_hot_encode(skills):
    encoding = [0, 0, 0]
    for s in skills:
        if s == 'A':
            encoding[0] = 1
        elif s == 'B':
            encoding[1] = 1
        elif s == 'C':
            encoding[2] = 1
    return encoding

def construct_input_vector(row):
    vector = []
    for t in range(1, 11):
        task_id = f"T{t}"
        emp_id = row[task_id]

        task = tasks[task_id]
        emp = employees[emp_id]

        task_features = [
            task["estimated_time"],
            task["difficulty"],
            task["deadline"]
        ] + one_hot_encode([task["skill_required"]])

        emp_features = [
            emp["hours_avail"],
            emp["skill_level"]
        ] + one_hot_encode(emp["skills"])

        vector.extend(task_features + emp_features)

    return np.array(vector).reshape(-1, 1)  # Shape: (110, 1)

## Model Definitions

In [ ]:
class NeuralNetwork :
  def __init__ (self , layer_dims , activation =’relu’):
  ...
  def forward (self , x):
  ...
  def backward (self , x, y_true ):
  ...
  def update_params (self , lr):
  ...

## Training Loop